# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
# print current directory
print(os.getcwd())

C:\Users\praka\Desktop\springboard\chapter6\guided_capstone


**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [2]:
df=pd.read_csv('data/step3_output.csv')
df.head()

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,Alaska,3939,2500,1,0.0,0,2,2,0,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,0
1,Eaglecrest Ski Area,Alaska,2600,1540,0,0.0,0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,0
2,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,0,1,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,0
3,Arizona Snowbowl,Arizona,11500,2300,0,0.0,1,0,2,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,1
4,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,2,3,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [3]:
# creating dummies for state, drop first=True to get rid of dummy variable trap
dummies=pd.get_dummies(df['state'],drop_first=True) 
df=pd.concat([df.drop('state',axis=1),dummies],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 58 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               328 non-null    object 
 1   summit_elev        328 non-null    int64  
 2   vertical_drop      328 non-null    int64  
 3   trams              328 non-null    int64  
 4   fastEight          328 non-null    float64
 5   fastSixes          328 non-null    int64  
 6   fastQuads          328 non-null    int64  
 7   quad               328 non-null    int64  
 8   triple             328 non-null    int64  
 9   double             328 non-null    int64  
 10  surface            328 non-null    int64  
 11  Runs               328 non-null    float64
 12  TerrainParks       328 non-null    float64
 13  LongestRun_mi      328 non-null    float64
 14  SkiableTerrain_ac  328 non-null    float64
 15  Snow Making_ac     328 non-null    float64
 16  daysOpenLastYear   328 non

In [4]:
df.head()

,Name,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Alyeska Resort,3939,2500,1,0.0,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Eaglecrest Ski Area,2600,1540,0,0.0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
2,Hilltop Ski Area,2090,294,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Arizona Snowbowl,11500,2300,0,0.0,1,0,2,2,1,...,0,0,0,0,0,0,0,0,0,0
4,Sunrise Park Resort,11100,1800,0,0.0,0,1,2,3,1,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [5]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.loc[:,'AdultWeekend']

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X)

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [6]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=101)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [7]:
#all first model set

from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error

lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [8]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [9]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

explained_variance_score(y_test, y_pred)

0.808470349631156

In [10]:
mean_absolute_error(y_test, y_pred)

6.0973183007046226

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [11]:
lm.intercept_

64.17843236505267

In [12]:
lm.coef_

array([ 2.45101115e+00,  6.22717328e-01,  2.10148591e-01, -1.12398341e-01,
        3.52493580e-01,  1.48046774e+00,  1.23735904e+00,  1.53144769e+00,
        6.80283794e-01, -3.98347500e-01, -3.91559922e-01,  4.02460335e-01,
       -3.73581968e-01, -1.16296610e+00, -1.83022396e-01,  5.03151066e-01,
        1.54632871e-01,  7.29260744e-01,  2.00682993e+01, -5.15083942e-02,
        1.58884266e+00, -7.02166254e-01, -6.34462666e-01, -1.30347651e+00,
       -1.83106469e+00,  6.41495443e-01, -1.35029525e+00, -5.60968282e-02,
       -3.11162128e-01, -5.07201208e-01,  1.16318578e+00,  5.39408372e-01,
        1.27809309e+00, -2.65237998e-01, -2.92444902e-01, -5.48384796e-01,
       -1.50601448e+00, -9.19802067e-02,  1.07136443e+00, -8.88178420e-15,
       -1.40742712e+00, -6.13893749e-02,  1.86366137e+00, -7.27951044e-01,
       -8.70856432e-01,  9.33365115e-01,  2.83117028e-01, -4.58394970e-01,
        1.23123901e+00, -5.04838004e-01, -1.17735254e-02,  1.30551082e+00,
       -8.24078595e-01,  

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [13]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
#pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])

pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values('Coefficient', ascending=False).head(10)

,Coefficient
AdultWeekday,20.068299
summit_elev,2.451011
North Carolina,1.863661
Colorado,1.831065
NightSkiing_ac,1.588843
triple,1.531448
Montana,1.506014
fastQuads,1.480468
New Mexico,1.407427
Idaho,1.350295


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [14]:
df_noState=pd.read_csv('data/step3_output.csv').drop('state', axis=1)
df_noState.head()

,Name,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,3939,2500,1,0.0,0,2,2,0,0,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,0
1,Eaglecrest Ski Area,2600,1540,0,0.0,0,0,0,0,4,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,0
2,Hilltop Ski Area,2090,294,0,0.0,0,0,0,1,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,0
3,Arizona Snowbowl,11500,2300,0,0.0,1,0,2,2,1,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,1
4,Sunrise Park Resort,11100,1800,0,0.0,0,1,2,3,1,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,1


In [15]:
from sklearn import preprocessing

X = df_noState.drop(['Name','AdultWeekend'], axis=1)
y =df_noState.loc[:,'AdultWeekend']
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X)

  
from sklearn.model_selection import train_test_split
y = y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=101)

In [16]:
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error

lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

y_pred = model.predict(X_test)

In [17]:
explained_variance_score(y_test, y_pred)

0.8172206707226075

In [18]:
mean_absolute_error(y_test, y_pred)

6.084592582801449

In [19]:
lm.intercept_

64.08422316852169

In [20]:
lm.coef_

array([-5.50996035e-01,  1.87214303e+00, -1.68869221e-01, -7.22355497e-02,
        4.96623014e-01,  1.52079514e+00,  1.32218483e+00,  1.36299186e+00,
        5.08404359e-01, -7.44705509e-01, -4.89404757e-01, -1.39076280e-01,
       -1.62912479e-01, -1.40599291e+00, -3.63984072e-02,  1.41370168e+00,
       -1.48303301e-02, -5.28023342e-01,  2.03238109e+01, -4.38089999e-01,
        1.42457097e+00, -1.39395982e+00])

In [21]:
coeff_df=pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values('Coefficient', ascending=False).head(10)
coeff_df

,Coefficient
AdultWeekday,20.323811
vertical_drop,1.872143
fastQuads,1.520795
NightSkiing_ac,1.424571
daysOpenLastYear,1.413702
SkiableTerrain_ac,1.405993
clusters,1.393960
triple,1.362992
quad,1.322185
surface,0.744706


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [22]:
df_no=pd.read_csv('data/step3_output.csv').drop(['state','summit_elev', 'vertical_drop', 'averageSnowfall','yearsOpen','daysOpenLastYear','projectedDaysOpen'], axis=1)
df_no.head()


,Name,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,Runs,TerrainParks,LongestRun_mi,SkiableTerrain_ac,Snow Making_ac,AdultWeekday,AdultWeekend,NightSkiing_ac,clusters
0,Alyeska Resort,1,0.0,0,2,2,0,0,2,76.0,2.0,1.0,1610.0,113.0,65.0,85.0,550.0,0
1,Eaglecrest Ski Area,0,0.0,0,0,0,0,4,0,36.0,1.0,2.0,640.0,60.0,47.0,53.0,0.0,0
2,Hilltop Ski Area,0,0.0,0,0,0,1,0,2,13.0,1.0,1.0,30.0,30.0,30.0,34.0,30.0,0
3,Arizona Snowbowl,0,0.0,1,0,2,2,1,2,55.0,4.0,2.0,777.0,104.0,89.0,89.0,0.0,1
4,Sunrise Park Resort,0,0.0,0,1,2,3,1,0,65.0,2.0,1.2,800.0,80.0,74.0,78.0,80.0,1


In [23]:
from sklearn import preprocessing

X = df_no.drop(['Name','AdultWeekend'], axis=1)
y =df_no.loc[:,'AdultWeekend']
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X)
  
from sklearn.model_selection import train_test_split
y = y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=101)

In [24]:
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error

lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

y_pred = model.predict(X_test)

In [25]:
explained_variance_score(y_test, y_pred)

0.8167326059237587

In [26]:
mean_absolute_error(y_test, y_pred)

6.017687740081119

In [27]:
lm.intercept_

64.08253991784082

In [28]:
lm.coef_

array([ 0.12926069, -0.10497161,  0.66330679,  1.61932041,  1.2922573 ,
        1.31312547,  0.52256502, -0.78903778,  0.20959956, -0.02908583,
        0.15109414, -1.51179451,  0.11048319, 20.70134093,  1.44665126,
       -1.45853377])

In [29]:
coeff_df=pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values('Coefficient', ascending=False).head(10)
coeff_df

,Coefficient
AdultWeekday,20.701341
fastQuads,1.619320
SkiableTerrain_ac,1.511795
clusters,1.458534
NightSkiing_ac,1.446651
triple,1.313125
quad,1.292257
surface,0.789038
fastSixes,0.663307
double,0.522565


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0.808 |6.097  |-|
| Model 2. | 0.8172|6.084 |'state'|
| Model 3. | 0.8167 |6.017 |'state','summit_elev', 'vertical_drop', 'averageSnowfall', 'yearsOpen', 'daysOpenLastYear', 'projectedDaysOpen'|

Model Selection:

**Model 3**- This excludes inherent features that the resort cannot manage.